## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pickle

from random import sample

from fastprogress.fastprogress import progress_bar
    
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, image_dataset_from_directory, img_to_array

from keras.applications.vgg16 import preprocess_input, VGG16
from keras.applications.vgg19 import VGG19 
from keras.models import Model

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score


## Model generation

#### Adapted from https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1

In [ ]:
# load images from folder

def load_images(image_directory):

    images_list = []
    
    with os.scandir(image_directory) as files:
        for file in files:
            if file.name.endswith('.png'):
                images_list.append(file.name)
                
    if os.path.exists('data')==False:
        os.makedirs('data')
    pd.DataFrame(images_list).to_csv('data/images_list.csv', index=True)
    
    return images_list

In [ ]:
# function to extract features from the images

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    
    return features

In [ ]:
# use a vgg model to extract features from images

def vgg_model(image_directory, images_list, model, model_name):
    if os.path.exists('model')==False:
        os.makedirs('model')
    
    vgg_model = model()
    vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)

    data = {}
    #p = filepath

    # loop through each image in the dataset
    for image in progress_bar(images_list):
        # try to extract the features and update the dictionary
        try:
            feature = extract_features(f'{image_directory}/{image}', vgg_model)
            data[image] = feature
    # if something fails, save the extracted features as a pickle file (optional)
        except:
            print('except')
            with open('model','wb') as file:
                pickle.dump(data, file)
    model_features = np.array(list(data.values()))
    model_features = model_features.reshape(-1,4096)
    
    pd.DataFrame(model_features).to_csv(f'model/{model_name}_features.csv', index=False)
    
    with open(f"model/{model_name}_model.pkl", "wb") as f:
        pickle.dump(vgg_model, f)
    
    return model_features #,model_filenames

## Clustering and cluster analysis

In [ ]:
# use PCA to reduce number of features
# plot explained variance to choose number of components for future analysis

def model_pca_components(model_features, model_name):
    pca = PCA(n_components=100, random_state=22)
    pca_features = pca.fit_transform(model_features)

    fig = plt.plot(pca.explained_variance_ratio_)
    plt.xlabel("Principal Component")
    plt.ylabel("Variance")
    plt.title('VGG19 PCA Explained Variance')
    plt.savefig(f'model/{model_name}_pca_variance.png', bbox_inches='tight')


In [ ]:
# extract PCA features using minimum number of components

def extract_pca_features(model_features, model_name, n_components=20):
    pca = PCA(n_components=n_components, random_state=22)
    pca_features = pca.fit_transform(model_features)
    
    if os.path.exists('model')==False:
        os.makedirs('model')
    pd.DataFrame(pca_features).to_csv(f'model/{model_name}_pca_features.csv', index=False)
    
    return pca_features


In [ ]:
# plot to see which value for k might be the best 

def n_cluster_analysis(min_k, max_k, pca_features, model_name):
    sse = []
    list_k = list(range(min_k, max_k))

    for k in list_k:
        km = KMeans(n_init = 'auto', n_clusters=k, random_state=22)
        km.fit(pca_features)
    
        sse.append(km.inertia_)

    # Plot sse against k
    plt.figure(figsize=(6, 6))
    plt.plot(list_k, sse)
    plt.xlabel(r'Number of clusters *k*')
    plt.ylabel('Sum of squared distance')
    
    if os.path.exists('model')==False:
        os.makedirs('model')
    plt.savefig(f'model/{model_name}_kmeans_inertia.png', bbox_inches='tight')


In [ ]:
# Adapted from https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#
# plot silhouette graphs to analyze and choose the proper number of clusters

def silhouette_analysis(pca_features, model_name, range_n_clusters):

    X = pca_features
    range_n_clusters = range_n_clusters
    kmeans_dict = {}

    for n_clusters in range_n_clusters:
        # Create a subplot with 1 row and 2 columns
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(18, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=22)
        cluster_labels = clusterer.fit_predict(X)
        
        if os.path.exists('model')==False:
            os.makedirs('model')
        pd.DataFrame(cluster_labels).to_csv(f'model/{model_name}_kmeans{n_clusters}.csv', index=False)       

        kmeans_dict[n_clusters] = cluster_labels

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print(
            "For n_clusters =",
            n_clusters,
            "The average silhouette_score is :",
            silhouette_avg,
        )

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(
                np.arange(y_lower, y_upper),
                0,
                ith_cluster_silhouette_values,
                facecolor=color,
                edgecolor=color,
                alpha=0.7,
            )

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(
            X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
        )

        # Labeling the clusters
        centers = clusterer.cluster_centers_
        # Draw white circles at cluster centers
        ax2.scatter(
            centers[:, 0],
            centers[:, 1],
            marker="o",
            c="white",
            alpha=1,
            s=200,
            edgecolor="k",
        )

        for i, c in enumerate(centers):
            ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

        ax2.set_title("The visualization of the clustered data.")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(
            "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
            % n_clusters,
            fontsize=14,
            fontweight="bold",
        )
        plt.savefig(f'model/{model_name}_kmeans{n_clusters}_silhouette.png', bbox_inches='tight')
        
    return kmeans_dict

In [ ]:
# display samples of each cluster to ensure proper groupings and to be able to assign grade label

def display_image_clusters(kmeans_dict, model_name, images_list):
    
    for n_cluster in kmeans_dict.keys():
        print("Number of clusters: ", n_cluster)
        groups = {}
        
        for file, cluster in zip(images_list, kmeans_dict[n_cluster]):
            if cluster not in groups.keys():
                groups[cluster] = []
                groups[cluster].append(file)
            else:
                groups[cluster].append(file)

        for cluster in range(len(groups)):
            print("Cluster: ", cluster)
            plt.figure(figsize = (25,25));

            # gets the list of filenames for a cluster
            files = groups[cluster]

            # only allow up to 100 images to be shown at a time
            if len(files) > 100:
                files = sample(files,100)

            # plot each image in the cluster
            for index, file in enumerate(files):
                plt.subplot(10,20,index+1);
                img = load_img(f'images/{file}')
                img = np.array(img)
                plt.imshow(img)
                plt.axis('off')
                
            if os.path.exists('model')==False:
                os.makedirs('model')
            plt.savefig(f'model/{model_name}_kmeans{n_cluster}_cluster{cluster}.png', bbox_inches='tight')
            plt.show()


## Make VGG19 model

In [ ]:
images_list = load_images(image_directory='images')

In [ ]:
model_features = vgg_model(image_directory='images', images_list=images_list, model=VGG19, model_name='VGG19')

In [ ]:
model_pca_components(model_features=model_features, model_name='VGG19')

In [ ]:
pca_features = extract_pca_features(model_features=model_features, model_name='VGG19', n_components=20)

In [ ]:
n_cluster_analysis(min_k=3, max_k=50, pca_features=pca_features, model_name='VGG19')

In [ ]:
kmeans_dict = silhouette_analysis(pca_features=pca_features, model_name='VGG19', range_n_clusters=[10, 20, 30, 40, 50])

In [ ]:
display_image_clusters(kmeans_dict=kmeans_dict, model_name='VGG19', images_list=images_list)

## Make VGG16 model

In [ ]:
model_features = vgg_model(image_directory='images', images_list=images_list, model=VGG16, model_name='VGG16')

In [ ]:
model_pca_components(model_features=model_features, model_name='VGG16')

In [ ]:
pca_features = extract_pca_features(model_features=model_features, model_name='VGG16', n_components=20)

In [ ]:
n_cluster_analysis(min_k=3, max_k=50, pca_features=pca_features, model_name='VGG16')

In [ ]:
kmeans_dict = silhouette_analysis(pca_features=pca_features, model_name='VGG16', range_n_clusters=[10, 20, 30, 40, 50])

In [ ]:
display_image_clusters(kmeans_dict=kmeans_dict, model_name='VGG16', images_list=images_list)
